In [84]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [85]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
    """Given 2 numbers a and b this tool returns their product"""
    return a * b

In [86]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [87]:
print(multiply.name)
print(multiply.description)
print(multiply.args_schema)

multiply
Given 2 numbers a and b this tool returns their product
<class 'langchain_core.utils.pydantic.multiply'>


## Tool Binding

In [88]:
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)

In [89]:
llm_with_tools=llm.bind_tools([multiply])

## Tool Calling 

In [90]:
llm_with_tools.invoke('Hi how are you?')

AIMessage(content="I'm functioning properly, thanks for asking. What can I help you with today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 235, 'total_tokens': 253, 'completion_time': 0.02999901, 'completion_tokens_details': None, 'prompt_time': 0.015201336, 'prompt_tokens_details': None, 'queue_time': 0.049698454, 'total_time': 0.045200346}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--62aff703-a36b-4944-b129-a2f05c0591bc-0', usage_metadata={'input_tokens': 235, 'output_tokens': 18, 'total_tokens': 253})

In [91]:
query= HumanMessage('can you multiply 7 and 8 for me?')

In [92]:
messages= [query]

In [93]:
llm_with_tools.invoke('Can you multiply 7 and 8 for me?')

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'pgaebsg94', 'function': {'arguments': '{"a":7,"b":8}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 241, 'total_tokens': 260, 'completion_time': 0.025078032, 'completion_tokens_details': None, 'prompt_time': 0.013298351, 'prompt_tokens_details': None, 'queue_time': 0.052540199, 'total_time': 0.038376383}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--3ebd6279-73d1-4fda-981a-201a881359d9-0', tool_calls=[{'name': 'multiply', 'args': {'a': 7, 'b': 8}, 'id': 'pgaebsg94', 'type': 'tool_call'}], usage_metadata={'input_tokens': 241, 'output_tokens': 19, 'total_tokens': 260})

In [94]:
llm_with_tools.invoke('Can you multiply 7 and 8 for me?').tool_calls[0]

{'name': 'multiply',
 'args': {'a': 7, 'b': 8},
 'id': 'xjggc2xmh',
 'type': 'tool_call'}

In [95]:
result=llm_with_tools.invoke('Can you multiply 7 and 8 for me?')

In [96]:
messages.append(result)

## Tool Execution

In [97]:
multiply.invoke(result.tool_calls[0]['args'])

56

In [98]:
tool_result=multiply.invoke(result.tool_calls[0])

In [99]:
messages.append(tool_result)

In [100]:
messages

[HumanMessage(content='can you multiply 7 and 8 for me?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'fv1fhqmar', 'function': {'arguments': '{"a":7,"b":8}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 241, 'total_tokens': 260, 'completion_time': 0.025495993, 'completion_tokens_details': None, 'prompt_time': 0.013718644, 'prompt_tokens_details': None, 'queue_time': 0.052398595, 'total_time': 0.039214637}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--567a346b-77ff-458a-8d94-7e50fceaa77b-0', tool_calls=[{'name': 'multiply', 'args': {'a': 7, 'b': 8}, 'id': 'fv1fhqmar', 'type': 'tool_call'}], usage_metadata={'input_tokens': 241, 'output_tokens': 19, 'total_tokens': 260}),
 ToolMessage(content='56', name=

In [101]:
llm_with_tools.invoke(messages).content

'The result of multiplying 7 and 8 is 56.'